In [106]:
!pip install pandas xlsxwriter


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [108]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


ConnectionRefusedError: [Errno 111] Connection refused

In [86]:
spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_10M_50COLUMNS_Update")


DataFrame[]

In [87]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_10M_50COLUMNS_Update"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [88]:
from pyspark.sql.types import (
    DoubleType, FloatType, LongType, StructType, StructField, 
    StringType, IntegerType, DateType
)

# Define the schema with 50 columns based on the required data types
schema = StructType([
    # StructField("vendor_id", LongType(), True),  # INT
    # StructField("trip_id", LongType(), True),  # INT
    # StructField("trip_distance", FloatType(), True),  # FLOAT
    # StructField("fare_amount", DoubleType(), True),  # DOUBLE
    # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# ] + [
    # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
    StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
    StructField(f"extra_col_{i}", DateType(), True)  # DATE
    for i in range(50)
])

# Create an empty DataFrame with the schema
df = spark.createDataFrame([], schema)

# Create the Iceberg table
df.writeTo("demo.nyc.taxis_10M_50COLUMNS_Update").create()


In [89]:
df = spark.table("demo.nyc.taxis_10M_50COLUMNS_Update")
df.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [90]:
import time, csv
from pyspark.sql.functions import col, when, lit
from pyspark.sql import functions as F
import os

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_Update")
records_before_op = df.count()

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )

    rows = df.count()
    
    df.writeTo("demo.nyc.taxis_10M_50COLUMNS_Update").append()
    end = time.time() - st
    total_insertion_time += end

    details = {"time_taken": f"{end:.2f} sec", "Operation": f"Inserted {rows} records", "records_after_op": records_before_op + rows}
    records_before_op += rows

    print(f"Inserted {rows} records in {end:.2f} sec.")

# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_Update")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

st = time.time()

# Updating `extra_col_1` (integer)
updated_df = df.withColumn(
    "extra_col_1",
    when(col("extra_col_0").isin(sampled_ids), col("extra_col_1") + 10)
    .otherwise(col("extra_col_1"))
)

# Updating `extra_col_3` (date) - Commented out
# updated_df = updated_df.withColumn(
#     "extra_col_3",
#     when(col("extra_col_0").isin(sampled_ids), F.date_add(col("extra_col_3"), 5))
#     .otherwise(col("extra_col_3"))
# )

# Overwrite updated data
updated_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_Update").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_10_1740401457.66906.parquet


Inserted 1000000 records in 38.94 sec.
Started with file=records_1000000_part_1_1740398687.6853974.parquet


Inserted 1000000 records in 32.78 sec.
Started with file=records_1000000_part_2_1740398997.7710938.parquet


Inserted 1000000 records in 32.41 sec.
Started with file=records_1000000_part_3_1740399303.6597402.parquet


...

Enter update percentage (e.g., 1 for 1%):  1


Updating 100000 rows (~1.0%)...


25/02/25 17:41:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/25 17:43:21 WARN DAGScheduler: Broadcasting large task bin

Py4JError: py4j does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [77]:
spark.sql("SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.history")


DataFrame[made_current_at: timestamp, snapshot_id: bigint, parent_id: bigint, is_current_ancestor: boolean]

In [84]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_1M_50COLUMNS.history ORDER BY made_current_at DESC")

df.show(

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
|2861666654166238235|2025-02-25 13:10:...|
|7867670594422745254|2025-02-25 13:09:...|
+-------------------+--------------------+



In [ ]:
WITH current_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(2861666654166238235)
),
previous_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(7867670594422745254)
)
SELECT p.*, c.*
FROM previous_snapshot p
FULL OUTER JOIN current_snapshot c
ON p.primary_key_column = c.primary_key_column
WHERE p.primary_key_column IS NULL OR c.primary_key_column IS NULL;


In [85]:
spark.sql("DESCRIBE demo.nyc.taxis_1M_50COLUMNS").show()


+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
| extra_col_0|   string|   NULL|
| extra_col_1|      int|   NULL|
| extra_col_2|   string|   NULL|
| extra_col_3|     date|   NULL|
| extra_col_4|   string|   NULL|
| extra_col_5|      int|   NULL|
| extra_col_6|   string|   NULL|
| extra_col_7|     date|   NULL|
| extra_col_8|   string|   NULL|
| extra_col_9|      int|   NULL|
|extra_col_10|   string|   NULL|
|extra_col_11|     date|   NULL|
|extra_col_12|   string|   NULL|
|extra_col_13|      int|   NULL|
|extra_col_14|   string|   NULL|
|extra_col_15|     date|   NULL|
|extra_col_16|   string|   NULL|
|extra_col_17|      int|   NULL|
|extra_col_18|   string|   NULL|
|extra_col_19|     date|   NULL|
+------------+---------+-------+
only showing top 20 rows

